In [282]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train.csv', 'gender_submission.csv', 'test.csv']


In [283]:
train_data= pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')
gender_submisson = pd.read_csv('../input/gender_submission.csv')

In [284]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [285]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [286]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


There are null values in Age,cabin,Embarked. Lets replace null values in Age with mean, Embarked with Mean and remove cabin

In [287]:
# Update sex column to numerical

train_data['Sex'] = train_data['Sex'].apply(lambda x: 1 if x=='male' else 0)
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


In [288]:
test_data['Sex'] = test_data['Sex'].apply(lambda x: 1 if x=='male' else 0)

In [289]:
#print(train_data['Age'].values)
mean_age = train_data.Age.mean()
train_data['Age'].fillna(value = mean_age, inplace = True)

In [290]:
#print(train_data['Age'].values)
mean_age = test_data.Age.mean()
test_data['Age'].fillna(value = mean_age, inplace = True)

In [291]:
print(train_data['Age'].values)

[22.         38.         26.         35.         35.         29.69911765
 54.          2.         27.         14.          4.         58.
 20.         39.         14.         55.          2.         29.69911765
 31.         29.69911765 35.         34.         15.         28.
  8.         38.         29.69911765 19.         29.69911765 29.69911765
 40.         29.69911765 29.69911765 66.         28.         42.
 29.69911765 21.         18.         14.         40.         27.
 29.69911765  3.         19.         29.69911765 29.69911765 29.69911765
 29.69911765 18.          7.         21.         49.         29.
 65.         29.69911765 21.         28.5         5.         11.
 22.         38.         45.          4.         29.69911765 29.69911765
 29.         19.         17.         26.         32.         16.
 21.         26.         32.         25.         29.69911765 29.69911765
  0.83       30.         22.         29.         29.69911765 28.
 17.         33.         16.         29.69

In [292]:
train_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [293]:
# Select the desired features
train_data = train_data[[ 'PassengerId', 'Survived', 'Pclass','Name','Sex', 'Age', 'SibSp', 'Parch', 'Ticket','Fare'] ]
# Select the desired features
test_data = test_data[[ 'PassengerId', 'Pclass','Name','Sex', 'Age', 'SibSp', 'Parch', 'Ticket','Fare'] ]

In [294]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500


In [295]:
# Create a Upper column
train_data['Upper'] = train_data['Pclass'].apply(lambda x: 1 if x==1 else 0)

# Create a Middle column
train_data['Middle'] = train_data['Pclass'].apply(lambda x: 1 if x==2 else 0)

# Create a Lower column
train_data['Lower'] = train_data['Pclass'].apply(lambda x: 1 if x==3 else 0)

In [296]:
# Create a Upper column
test_data['Upper'] = test_data['Pclass'].apply(lambda x: 1 if x==1 else 0)

# Create a Middle column
test_data['Middle'] = test_data['Pclass'].apply(lambda x: 1 if x==2 else 0)

# Create a Lower column
test_data['Lower'] = test_data['Pclass'].apply(lambda x: 1 if x==3 else 0)

In [297]:
train_data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Upper          0
Middle         0
Lower          0
dtype: int64

In [298]:
# Select the desired features
selected_features = ['Sex', 'Age', 'Upper', 'Middle', 'Lower', 'SibSp', 'Parch', 'Fare'] #

# Select the desired features
features = train_data[selected_features]
survival = train_data['Survived']

In [299]:
features.head()

,Sex,Age,Upper,Middle,Lower,SibSp,Parch,Fare
0,1,22.0,0,0,1,1,0,7.2500
1,0,38.0,1,0,0,1,0,71.2833
2,0,26.0,0,0,1,0,0,7.9250
3,0,35.0,1,0,0,1,0,53.1000
4,1,35.0,0,0,1,0,0,8.0500


In [300]:
# Perform train, test, split

train_features, test_features, train_labels, test_labels = train_test_split(features, survival, train_size = 0.8)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [301]:
test_data1 = test_data[selected_features]


In [302]:
test_data1.isnull().sum()

Sex       0
Age       0
Upper     0
Middle    0
Lower     0
SibSp     0
Parch     0
Fare      1
dtype: int64

In [303]:
#print(train_data['Age'].values)
mean_fare = test_data1.Fare.mean()
test_data1['Fare'].fillna(value = mean_fare, inplace = True)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [304]:
# Scale the feature data so it has mean = 0 and standard deviation = 1
normalize = StandardScaler()
train_features = normalize.fit_transform(train_features)
test_features = normalize.transform(test_features)
test_data1 =normalize.transform(test_data1) 

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """


In [305]:
# Create and train the model
model = LogisticRegression()
model.fit(train_features, train_labels)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [306]:
# Score the model on the train data
print(model.score(train_features, train_labels))

0.8019662921348315


In [307]:
# Score the model on the test data
print(model.score(test_features, test_labels))

0.8044692737430168


In [308]:
print(list(zip(selected_features, model.coef_[0])))

[('Sex', -1.2732380942521757), ('Age', -0.4485672639199203), ('Upper', 0.4421957280586067), ('Middle', 0.05773844492915665), ('Lower', -0.4245908054687876), ('SibSp', -0.29630759267397294), ('Parch', -0.15837464440687885), ('Fare', 0.18162972540961697)]


In [309]:
prdctions=model.predict_proba(test_data1)[:,1]
print(prdctions)

[0.09281854 0.42332923 0.09671075 0.11771267 0.59764028 0.17403036
 0.63519627 0.20401305 0.72573863 0.09258757 0.10607103 0.32795966
 0.93690641 0.07741467 0.85610017 0.82784643 0.21755284 0.14071159
 0.59726425 0.50704978 0.23496411 0.16900412 0.92626066 0.52349255
 0.86933137 0.04457375 0.9392702  0.13448334 0.37133569 0.06813435
 0.11680344 0.20420965 0.46286538 0.48920932 0.42276321 0.15162822
 0.63332947 0.70541168 0.12550408 0.12355531 0.0687372  0.45882327
 0.07536219 0.82846652 0.86088897 0.12489241 0.33865097 0.10602202
 0.83868241 0.49903867 0.47664554 0.270813   0.7752622  0.90491048
 0.2491607  0.06902295 0.09137522 0.12478587 0.08557062 0.95883781
 0.15882061 0.23668895 0.15413348 0.69738629 0.59019608 0.83112564
 0.72619715 0.33308379 0.45425586 0.78388269 0.68243375 0.14099944
 0.64350726 0.47529623 0.95270037 0.60175678 0.10612288 0.75231123
 0.24923439 0.68243375 0.15297847 0.26412161 0.30525109 0.10607103
 0.24594349 0.08511507 0.65936126 0.72631756 0.6330824  0.3675

In [310]:
print(gender_submisson.PassengerId)

0       892
1       893
2       894
3       895
4       896
5       897
6       898
7       899
8       900
9       901
10      902
11      903
12      904
13      905
14      906
15      907
16      908
17      909
18      910
19      911
20      912
21      913
22      914
23      915
24      916
25      917
26      918
27      919
28      920
29      921
       ... 
388    1280
389    1281
390    1282
391    1283
392    1284
393    1285
394    1286
395    1287
396    1288
397    1289
398    1290
399    1291
400    1292
401    1293
402    1294
403    1295
404    1296
405    1297
406    1298
407    1299
408    1300
409    1301
410    1302
411    1303
412    1304
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64


In [311]:
my_submission = pd.DataFrame({'PassengerId': gender_submisson.PassengerId, 'Survived': prdctions})

In [312]:
my_submission = [('PassengerId', gender_submisson.PassengerId),('Survived', model.predict_proba(test_features))      ]

my_submission = pd.DataFrame(my_submission)
my_submission.to_csv('my_submission.csv')
